# Recommendations with MovieTweetings: Knowledge Based Recommendation

We'll use the most simple recommendation mechanism, that is to recommend the most popular items.
To get started, read in the libraries and the two datasets that will be used throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Read in the datasets
movies  = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [2]:
print(movies.shape)
print(reviews.shape)

(31245, 35)
(712337, 23)


In [3]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [4]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


## 1. Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [5]:
def movie_rating (reviews, movies):
    movie_rating = reviews.groupby("movie_id")["rating"]
    
    avg_rating   = movie_rating.mean().reset_index(name="avg")
    count_rating = movie_rating.count().reset_index(name="count")
    most_recent  = reviews.groupby("movie_id")["date"].max().reset_index(name="most_recent")
    
    movies = movies.merge(avg_rating,  on="movie_id", how="left")
    movies = movies.merge(count_rating,on="movie_id", how="left")
    movies = movies.merge(most_recent, on="movie_id", how="left")

    ranked_movies = movies.sort_values(by=["avg","count","most_recent"],
                           ascending=False)
    
    return ranked_movies
    
def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top   - an integer of the number recommendations you want back
    
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    
    top_n_movies = ranked_movies[ranked_movies["count"]>=5].iloc[:n_top]["movie"]
    
    
    return list(top_n_movies)

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [6]:
ranked_movies = movie_rating(reviews, movies)
ranked_movies.shape

(31245, 38)

In [8]:
ranked_movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Action,Documentary,Animation,Comedy,Short,Western,Thriller,avg,count,most_recent
28837,4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy,2015,0,0,1,0,0,0,...,0,0,0,1,0,0,0,10.0,48,2016-08-14 17:16:50
29398,5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,...,0,0,0,1,1,0,0,10.0,28,2016-01-08 00:44:43
29894,5688932,Sorry to Bother You (2018),Comedy|Fantasy|Sci-Fi,2018,0,0,1,0,0,0,...,0,0,0,1,0,0,0,10.0,14,2018-06-17 01:44:48
24083,2737018,Selam (2013),Drama|Romance,2013,0,0,1,0,0,0,...,0,0,0,0,0,0,0,10.0,10,2015-05-10 22:56:01
23613,2560840,"Quiet Riot: Well Now You're Here, There's No W...",Documentary|Music,2014,0,0,1,0,0,0,...,0,1,0,0,0,0,0,10.0,6,2016-01-23 00:30:44


In [7]:
# Top 20 movies recommended for id 1
recs_20_for_1 = popular_recommendations(1,20,ranked_movies)

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968,5,ranked_movies)

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000,100,ranked_movies)

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43,35,ranked_movies)

In [9]:
recs_20_for_1

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Third Contact (2011)',
 'Romeo Juliet (2009)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Agnelli (2017)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)']

**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


## 2. Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [ ]:
def movie_filter(user_id,n_top,years,genres,ranked_movies):
    ranked_movies = ranked_movies[ranked_movies['year'].isin(years)]
    
    num_genre_match = ranked_movies[genres].sum(axis=1)
    ranked_movies = ranked_movies.loc[num_genre_match>0,:]
    
    top_movies = list(ranked_movies['movie'][:n_top])